<a href="https://colab.research.google.com/github/Blackman9t/Advanced-Data-Science/blob/master/pyspark_fundamentals_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBM intro to spark lab, part 3 and DataCamp intro to Pyspark, lessons 3 and 4.

**This is a very comprehensive notebook on pyspark. It's a sequel to the notebook pyspark_fundamentals_1. This notebook contains tutorials from IBM and DataCamp combined.<br>
This notebook holds the IBM intro to spark tutorials part 3 and the DataCamp intro to pyspark course lessons 3 and 4.<br> A continuation link is provided at the end of this notebook**

First let's load spark dependencies to run in colab

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

     |████████████████████████████████| 217.8MB 63kB/s 
     |████████████████████████████████| 204kB 51.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=28fe30c55ad6ee57152058768afa5196684b2277a3bb4e0e903309962dd3b781
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


Next let's instantiate a SparkContext object to connect to the spark cluster if none exists

In [2]:
from pyspark import SparkConf, SparkContext

try:
    conf = SparkConf().setMaster('local').setAppName('myApp')
    sc = SparkContext(conf=conf)
    print('SparkContext Initialised successfully!')
except Exception as e:
    print(e)

SparkContext Initialised successfully!


In [3]:
# Let's see the SparkContext Object
sc

<SparkContext master=local appName=myApp>

Next let's create a SparkSession as our interface to the SparkContect we created above

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('myApp').getOrCreate()

In [5]:
# Let's see the SparkSession
spark

In [0]:
# Let's import other possible libraries we may use
import pandas as pd
import numpy as np

## DataCamp, Course 1 Contd..:
**Lesson 3: Getting started with machine learning pipelines**

________________________________________________________________________________

At the core of the pyspark.ml module are the Transformer and Estimator classes. Almost every other class in the module behaves similarly to these two basic classes.

Transformer classes have a .transform() method that takes a DataFrame and returns a new DataFrame; usually the original one with a new column appended. For example, you might use the class Bucketizer to create discrete bins from a continuous feature or the class PCA to reduce the dimensionality of your dataset using principal component analysis.

Estimator classes all implement a .fit() method. These methods also take a DataFrame, but instead of returning another DataFrame they return a model object. This can be something like a StringIndexerModel for including categorical data saved as strings in your models, or a RandomForestModel that uses the random forest algorithm for classification or regression.

First let's grab our Flights, Airports and Planes data sets using Wget and then read these into Spark DataFrames

In [7]:
! wget 'https://assets.datacamp.com/production/repositories/1237/datasets/fa47bb54e83abd422831cbd4f441bd30fd18bd15/flights_small.csv'
! wget 'https://assets.datacamp.com/production/repositories/1237/datasets/6e5c4ac2a4799338ba7e13d54ce1fa918da644ba/airports.csv'
! wget 'https://assets.datacamp.com/production/repositories/1237/datasets/231480a2696c55fde829ce76d936596123f12c0c/planes.csv'

--2020-03-20 00:25:47--  https://assets.datacamp.com/production/repositories/1237/datasets/fa47bb54e83abd422831cbd4f441bd30fd18bd15/flights_small.csv
Resolving assets.datacamp.com (assets.datacamp.com)... 99.84.216.127, 99.84.216.5, 99.84.216.40, ...
Connecting to assets.datacamp.com (assets.datacamp.com)|99.84.216.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 614174 (600K)
Saving to: ‘flights_small.csv’

flights_small.csv   100%[===================>] 599.78K  --.-KB/s    in 0.007s  

2020-03-20 00:25:47 (79.4 MB/s) - ‘flights_small.csv’ saved [614174/614174]

--2020-03-20 00:25:49--  https://assets.datacamp.com/production/repositories/1237/datasets/6e5c4ac2a4799338ba7e13d54ce1fa918da644ba/airports.csv
Resolving assets.datacamp.com (assets.datacamp.com)... 99.84.216.127, 99.84.216.5, 99.84.216.40, ...
Connecting to assets.datacamp.com (assets.datacamp.com)|99.84.216.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84548 (83K)

In [8]:
flights = spark.read.csv('flights_small.csv', header=True)
flights.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
+----+-----+---+--------+---------+-----

In [9]:
airports = spark.read.csv('airports.csv',header=True)
airports.show(5)

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
|06N|     Randall Airport| 41.431912|-74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|31.0744722|-81.4277778|  11| -4|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 5 rows



In [10]:
planes = spark.read.csv('planes.csv',header=True)
planes.show(5)

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
only showing top 5 rows



**Join the DataFrames**

In the next two chapters you'll be working to build a model that predicts whether or not a flight will be delayed based on the flights data. This model will also include information about the plane that flew the route, so the first step is to join the two tables: flights and planes!

In [11]:
# First, rename the year column of planes to plane_year to avoid duplicate column names.
planes = planes.withColumnRenamed('year','plane_year')
planes.show(5)

+-------+----------+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|plane_year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----------+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|      1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
+-------+----------+--------------------+----------------+--------+-------+-----+-----+---------+
only showing top 5 rows



In [12]:
# Create a new DataFrame called model_data by joining the flights table with planes using the tailnum column as the key.
model_data = flights.join(planes, on='tailnum', how='left_outer')
model_data.show(5)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|      2011|Fixed wing multi ...|      AIRBUS|A320-214|      2|  182|   NA|Turbo-fan|
| N559AS|2014|    1| 22|    1040|        5|    1505|        5|     AS|   851|   SEA| HNL|     360|    2677|  10|    40|      2006|Fixed wing multi ...|      BOEING| 737-890|      2|  1

In [13]:
# Let's see how many rows exist
model_data.count()

10000

In [14]:
# let's print the schema and see how it's like
model_data.printSchema()

root
 |-- tailnum: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- plane_year: string (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: string (nullable = true)
 |-- seats: string (nullable = true)
 |-- speed: string (nullable = true)
 |-- engine: string (nullable = true)



**Data types**

Good work! Before you get started modeling, it's important to know that Spark only handles numeric data. That means all of the columns in your DataFrame must be either integers or decimals (called 'doubles' in Spark).

When we imported our data, we let Spark guess what kind of information each column held. Unfortunately, Spark doesn't always guess right and you can see that some of the columns in our DataFrame are strings containing numbers as opposed to actual numeric values.

To remedy this, you can use the .cast() method in combination with the .withColumn() method. It's important to note that .cast() works on columns, while .withColumn() works on DataFrames.

The only argument you need to pass to .cast() is the kind of value you want to create, in string form. For example, to create integers, you'll pass the argument "integer" and for decimal numbers you'll use "double".

You can put this call to .cast() inside a call to .withColumn() to overwrite the already existing column, just like you did in the previous chapter!

**String to integer**

Now you'll use the .cast() method you learned in the previous exercise to convert all the appropriate columns from your DataFrame model_data to integers!

To convert the type of a column using the .cast() method, you can write code like this:
```
dataframe = dataframe.withColumn("col", dataframe.col.cast("new_type"))
```



In [0]:
# Use the method .withColumn() to .cast() the following columns to type "integer". Access the columns using the df.col notation:
# model_data.arr_delay
# model_data.air_time
# model_data.month
# model_data.plane_year

model_data = model_data.withColumn("arr_delay", model_data.arr_delay.cast('integer'))
model_data = model_data.withColumn("air_time", model_data.air_time.cast('integer'))
model_data = model_data.withColumn("month", model_data.month.cast('integer'))
model_data = model_data.withColumn("plane_year", model_data.plane_year.cast('integer'))

In [16]:
# Let'see the schema data types again
model_data.printSchema()

root
 |-- tailnum: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- plane_year: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: string (nullable = true)
 |-- seats: string (nullable = true)
 |-- speed: string (nullable = true)
 |-- engine: string (nullable = true)



**Create a new column**

In the last exercise, you converted the column plane_year to an integer. This column holds the year each plane was manufactured. However, your model will use the planes' age, which is slightly different from the year it was made!

Create the column plane_age using the .withColumn() method and subtracting the year of manufacture (column plane_year) from the year (column year) of the flight.

In [0]:
model_data = model_data.withColumn('plane_age', model_data.year - model_data.plane_year)

In [18]:
model_data.show(5)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|plane_age|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|      2011|Fixed wing multi ...|      AIRBUS|A320-214|      2|  182|   NA|Turbo-fan|      3.0|
| N559AS|2014|    1| 22|    1040|        5|    1505|        5|     AS|   851|   SEA| HNL|     360|    2677|  10|    40|      2006|Fixed wing mul

**Making a Boolean**

Consider that you're modeling a yes or no question: is the flight late? However, your data contains the arrival delay in minutes for each flight. Thus, you'll need to create a boolean column which indicates whether the flight was late or not!

In [19]:
#Use the .withColumn() method to create the column is_late. This column is equal to model_data.arr_delay > 0.
model_data = model_data.withColumn('is_arrival_late', model_data.arr_delay > 0)
model_data.show(3)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+---------------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|plane_age|is_arrival_late|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+---------------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|      2011|Fixed wing multi ...|      AIRBUS|A320-214|      2|  182|   NA|Turbo-fan|      3.0|          false|
| N559AS|2014|    1| 22|    1040|        5|    1505|        5|     AS|   851|   

In [20]:
# Convert this column to an integer column so that you can use it in your model and name it label 
# (this is the default name for the response variable in Spark's machine learning routines).

model_data = model_data.withColumn('label', model_data.is_arrival_late.cast('integer'))
model_data.show(3)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+---------------+-----+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|plane_age|is_arrival_late|label|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+---------------+-----+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|      2011|Fixed wing multi ...|      AIRBUS|A320-214|      2|  182|   NA|Turbo-fan|      3.0|          false|    0|
| N559AS|2014|    1| 22|    1040|        5|    1505|    

**Remove missing values**

In [0]:
cols = model_data.columns

for col_name in cols:
    model_data = model_data.filter(col_name + ' is not Null')

In [22]:
# Let's see how many rows are left from the filter exercise above
model_data.count()

9303

**Strings and factors**

As you know, Spark requires numeric data for modeling. So far this hasn't been an issue; even boolean columns can easily be converted to integers without any trouble. But you'll also be using the airline and the plane's destination as features in your model. These are coded as strings and there isn't any obvious way to convert them to a numeric data type.

Fortunately, PySpark has functions for handling this built into the pyspark.ml.features submodule. You can create what are called 'one-hot vectors' to represent the carrier and the destination of each flight. A one-hot vector is a way of representing a categorical feature where every observation has a vector in which all elements are zero except for at most one element, which has a value of one (1).

Each element in the vector corresponds to a level of the feature, so it's possible to tell what the right level is by seeing which element of the vector is equal to one (1).

The first step to encoding your categorical feature is to create a StringIndexer. Members of this class are Estimators that take a DataFrame with a column of strings and map each unique string to a number. Then, the Estimator returns a Transformer that takes a DataFrame, attaches the mapping to it as metadata, and returns a new DataFrame with a numeric column corresponding to the string column.

The second step is to encode this numeric column as a one-hot vector using a OneHotEncoder. This works exactly the same way as the StringIndexer by creating an Estimator and then a Transformer. The end result is a column that encodes your categorical feature as a vector that's suitable for machine learning routines!

This may seem complicated, but don't worry! All you have to remember is that you need to create a StringIndexer and a OneHotEncoder, and the Pipeline will take care of the rest.

**Carrier column**

In this exercise you'll create a StringIndexer and a OneHotEncoder to code the carrier column. To do this, you'll call the class constructors with the arguments inputCol and outputCol.

The inputCol is the name of the column you want to index or encode, and the outputCol is the name of the new column that the Transformer should create.

In [23]:
# Create a StringIndexer called carr_indexer by calling StringIndexer() with inputCol="carrier" and outputCol="carrier_index".
from pyspark.ml.feature import StringIndexer

carr_indexer = StringIndexer(inputCol='carrier', outputCol='carrier_index')
# to immediately see the effect of the StringIndexer, lets say...
indexed = carr_indexer.fit(model_data).transform(model_data)  # This creates a new data frame
indexed.show(3)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+---------------+-----+-------------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|plane_age|is_arrival_late|label|carrier_index|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+---------------+-----+-------------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|      2011|Fixed wing multi ...|      AIRBUS|A320-214|      2|  182|   NA|Turbo-fan|      3.0|          false|    0|          7.0|


In [24]:
# Create a OneHotEncoder called carr_encoder by calling OneHotEncoder() with inputCol="carrier_index" and outputCol="carrier_fact".
from pyspark.ml.feature import OneHotEncoder

carr_encoder = OneHotEncoder(inputCol='carrier_index', outputCol='carrier_fact')
# to immediately see the effect of the OneHotEncoder, lets say...
encoded = carr_encoder.transform(indexed)  # This creates a new data frame. Note that encoder is a transformer and has no fit method
encoded.show(3)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+---------------+-----+-------------+--------------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|plane_age|is_arrival_late|label|carrier_index|  carrier_fact|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+---------------+-----+-------------+--------------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|      2011|Fixed wing multi ...|      AIRBUS|A320-214|      2|  182|   NA|Turbo-fan|  

**Destination Column**

Now you'll encode the dest column just like we did the carrier column.

In [0]:
# Create a StringIndexer called dest_indexer by calling StringIndexer() with inputCol="dest" and outputCol="dest_index".
dest_indexer = StringIndexer(inputCol='dest', outputCol='dest_index')

In [0]:
# Create a OneHotEncoder called dest_encoder by calling OneHotEncoder() with inputCol="dest_index" and outputCol="dest_fact".
dest_encoder = OneHotEncoder(inputCol='dest_index', outputCol='dest_fact')

**Assemble a vector**

The last step in the Pipeline is to combine all of the columns containing our features into a single column. This has to be done before modeling can take place because every Spark modeling routine expects the data to be in this form. You can do this by storing each of the values from a column as an entry in a vector. Then, from the model's point of view, every observation is a vector that contains all of the information about it and a label that tells the modeler what value that observation corresponds to.

Because of this, the pyspark.ml.feature submodule contains a class called VectorAssembler. This Transformer takes all of the columns you specify and combines them into a new vector column.

In [0]:
# Create a VectorAssembler by calling VectorAssembler() with the inputCols names as a list and the outputCol name "features".
# The list of columns should be ["month", "air_time", "carrier_fact", "dest_fact", "plane_age"].
from pyspark.ml.feature import VectorAssembler

vec_assembler = VectorAssembler(inputCols=["month", "air_time", "carrier_fact", "dest_fact", "plane_age"], outputCol='features')

**Create the pipeline**

You're finally ready to create a Pipeline!

Pipeline is a class in the pyspark.ml module that combines all the Estimators and Transformers that you've already created. This lets you reuse the same modeling process over and over again by wrapping it up in one simple object. Neat, right?

_Import Pipeline from pyspark.ml._

Call the Pipeline() constructor with the keyword argument stages to create a Pipeline called flights_pipe.<br>
stages should be a list holding all the stages you want your data to go through in the pipeline. Here this is just:
``` 
[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler]
```

In [0]:
from pyspark.ml import Pipeline
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler])

**Test vs Train**

After you've cleaned your data and gotten it ready for modeling, one of the most important steps is to split the data into a test set and a train set. After that, don't touch your test data until you think you have a good model! As you're building models and forming hypotheses, you can test them on your training data to get an idea of their performance.

Once you've got your favorite model, you can see how well it predicts the new data in your test set. This never-before-seen data will give you a much more realistic idea of your model's performance in the real world when you're trying to predict or classify new data.

In Spark it's important to make sure you split the data after all the transformations. This is because operations like StringIndexer don't always produce the same index even when given the same list of strings.



**Transform the data:**

Hooray, now you're finally ready to pass your data through the Pipeline you created!

In [0]:
# Create the DataFrame piped_data by calling the Pipeline methods .fit() and .transform() in a chain. 
# Both of these methods take model_data as their only argument.

piped_data = flights_pipe.fit(model_data).transform(model_data)

In [30]:
piped_data.show(3)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+---------------+-----+----------+---------------+-------------+--------------+--------------------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|plane_age|is_arrival_late|label|dest_index|      dest_fact|carrier_index|  carrier_fact|            features|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+---------------+-----+----------+---------------+-------------+--------------+--------------------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5

**Split the data**

Now that you've done all your manipulations, the last step before modeling is to split the data!

Use the DataFrame method .randomSplit() to split piped_data into two pieces, training with 75% of the data, and test with 25% of the data by passing the list [.75, .25] to the .randomSplit() method.

In [0]:
training, testing = piped_data.randomSplit([0.75, 0.25])

Let's see howmany rows in both training and testing sets

In [32]:
print('Training set has {}, while testing set has {} observations. Total is {} observations.'.format(training.count(), testing.count(), (training.count() + testing.count())))

Training set has 6972, while testing set has 2331 observations. Total is 9303 observations.


## DataCamp, Course 1 Contd...:
**Lesson 4: Model tuning and selection**

________________________________________________________________________________

**What is logistic regression?**

The model you'll be fitting in this chapter is called a logistic regression. This model is very similar to a linear regression, but instead of predicting a numeric variable, it predicts the probability (between 0 and 1) of an event.

To use this as a classification algorithm, all you have to do is assign a cutoff point to these probabilities. If the predicted probability is above the cutoff point, you classify that observation as a 'yes' (in this case, the flight being late), if it's below, you classify it as a 'no'!

You'll tune this model by testing different values for several hyperparameters. A hyperparameter is just a value in the model that's not estimated from the data, but rather is supplied by the user to maximize performance. For this course it's not necessary to understand the mathematics behind all of these values - what's important is that you'll try out a few different choices and pick the best one.

**Create the modeler**

The Estimator you'll be using is a LogisticRegression from the pyspark.ml.classification submodule.

In [33]:
# Import the LogisticRegression class from pyspark.ml.classification.
# Create a LogisticRegression called lr by calling LogisticRegression() with no arguments.

from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()
print(lr)

LogisticRegression_a096ec66d781


**Cross validation**

In the next few exercises you'll be tuning your logistic regression model using a procedure called k-fold cross validation. This is a method of estimating the model's performance on unseen data (like your test DataFrame).

It works by splitting the training data into a few different partitions. The exact number is up to you, but in this course you'll be using PySpark's default value of three. Once the data is split up, one of the partitions is set aside, and the model is fit to the others. Then the error is measured against the held out partition. This is repeated for each of the partitions, so that every block of data is held out and used as a test set exactly once. Then the error on each of the partitions is averaged. This is called the cross validation error of the model, and is a good estimate of the actual error on the held out data.

You'll be using cross validation to choose the hyperparameters by creating a grid of the possible pairs of values for the two hyperparameters, elasticNetParam and regParam, and using the cross validation error to compare all the different models so you can choose the best one!

Cross validation helps us to estimate the model error on the held-out testing data set

**Create the evaluator**

The first thing you need when doing cross validation for model selection is a way to compare different models. Luckily, the pyspark.ml.evaluation submodule has classes for evaluating different kinds of models. Your model is a binary classification model, so you'll be using the BinaryClassificationEvaluator from the pyspark.ml.evaluation module.

This evaluator calculates the area under the ROC. This is a metric that combines the two kinds of errors a binary classifier can make (false positives and false negatives) into a simple number. You'll learn more about this towards the end of the chapter!

In [0]:
# Import the submodule pyspark.ml.evaluation as evals.
# Create evaluator by calling evals.BinaryClassificationEvaluator() with the argument metricName="areaUnderROC".

import pyspark.ml.evaluation as evals

evaluator = evals.BinaryClassificationEvaluator(metricName='areaUnderROC')

**Make a grid**

Next, you need to create a grid of values to search over when looking for the optimal hyperparameters. The submodule pyspark.ml.tuning includes a class called ParamGridBuilder that does just that (maybe you're starting to notice a pattern here; PySpark has a submodule for just about everything!).

You'll need to use the .addGrid() and .build() methods to create a grid that you can use for cross validation. The .addGrid() method takes a model parameter (an attribute of the model Estimator, lr, that you created a few exercises ago) and a list of values that you want to try. The .build() method takes no arguments, it just returns the grid that you'll use later.

In [0]:
# Import the submodule pyspark.ml.tuning under the alias tune.
import pyspark.ml.tuning as tune

# Call the class constructor ParamGridBuilder() with no arguments. Save this as grid.
grid = tune.ParamGridBuilder()

# Call the .addGrid() method on grid with lr.regParam as the first argument and np.arange(0, .1, .01) as the second argument. 
# This second call is a function from the numpy module (imported as np) that creates a list of numbers from 0 to .1, incrementing by .01. Overwrite grid with the result.
# Add the hyperparameter
grid = grid.addGrid(lr.regParam, np.arange(0, .1, .01))

# Update grid again by calling the .addGrid() method a second time create a grid for lr.elasticNetParam that includes only the values [0, 1].
grid = grid.addGrid(lr.elasticNetParam, [0,1])

# Build the grid
grid = grid.build()

**Make the validator**

The submodule pyspark.ml.tuning also has a class called CrossValidator for performing cross validation. This Estimator takes the modeler you want to fit, the grid of hyperparameters you created, and the evaluator you want to use to compare your models.

The submodule pyspark.ml.tune has already been imported as tune. You'll create the CrossValidator by passing it the logistic regression Estimator lr, the parameter grid, and the evaluator you created in the previous exercises.
<br>Name this object cv.


In [0]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lr,
               estimatorParamMaps=grid,
               evaluator=evaluator)

**Fit the model(s)**

You're finally ready to fit the models and select the best one!

In [0]:
# Fit cross validation models
models = cv.fit(training)

# Extract the best model
best_lr = models.bestModel

In [38]:
print(best_lr)

LogisticRegressionModel: uid = LogisticRegression_a096ec66d781, numClasses = 2, numFeatures = 81


In [39]:
# We can also print the coefficient and intercept of the Logistic Regression model by using the following command:

#coefficient of the regression model
coeff = best_lr.coefficients

#X and Y intercept
intrcpt = best_lr.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intrcpt)

The coefficient of the model is : DenseVector([0.0229, 0.0767, 0.388, 1.0505, 0.3535, 0.5491, 0.2765, 0.236, 0.8576, -0.6089, 0.8991, 1.1357, -1.797, -4.1159, -4.4733, -3.4376, -5.314, -9.3034, -1.5738, -1.9482, -1.4171, -0.1077, -5.0248, -9.902, -13.9349, -7.5439, -11.6399, -4.8378, -5.1044, -15.9588, 2.9324, -20.8595, -15.9695, 2.6813, -3.7953, -3.7808, -16.6652, -11.9803, -9.7055, 2.9887, -21.0564, -15.2431, -11.213, -9.8323, -15.3228, -14.7779, -4.2681, -1.8568, -15.885, -6.245, 2.7179, -21.0493, -6.2261, -2.6879, -4.5536, -3.6426, -8.701, -21.5675, 2.6312, -15.6197, -10.8209, -17.2288, -18.2425, -4.9374, -7.113, -9.9699, -0.151, -12.843, -12.515, -9.4691, -18.1167, -13.1716, -77.4042, 2.3272, 1.4296, -3.3072, -12.4057, 5.2589, -18.3409, -2.1278, 0.0001])
The Intercept of the model is : -6.530859


**Evaluating binary classifiers**

For this course we'll be using a common metric for binary classification algorithms call the AUC, or area under the curve. In this case, the curve is the ROC, or receiver operating curve. The details of what these things actually measure isn't important for this course. All you need to know is that for our purposes, the closer the AUC is to one (1), the better the model is!

**Evaluate the model**

Remember the test data that you set aside waaaaaay back in chapter 3? It's finally time to test your model on it! You can use the same evaluator you made to fit the model.

In [0]:
# Use your model to generate predictions by applying best_lr.transform() to the test data. Save this as test_results.
test_results = best_lr.transform(testing)

In [41]:
# Call evaluator.evaluate() on test_results to compute the AUC. Print the output.
print(evaluator.evaluate(test_results))

0.6956451035620336


# DataCamp Course 2: Big Data Fundamentals with PySpark.

### Lesson 1: Introduction to Big Data analysis with Spark

**Inspecting The SparkContext**

SparkContext(sc) is an entry point into the world of spark functionality. An entry point is where control is transferred from operating system(os) to the provided program. in simpler terms, it's like a key to your house, without which there is no entry.
<br>Let's inspect some of the attributes of teh SparkContect.

In [42]:
# Let's print the SparkContext Version
sc.version

'2.4.5'

In [43]:
# Let's print the Python Version that sc runs on.
sc.pythonVer

'3.6'

In [44]:
# Let's print the Master. Master is the URL of the Cluster, or 'local' string to run in local mode.
# sc.master returns 'local', meaning the SparkContext acts as a master on a local node, using all available threads on the computer where it is running.
sc.master

'local'

**Loading Data in Pyspark:**

We can load raw data in pyspark using SparkContext by two distinct means:
<br>1. SparkContext parallelize() method on a list:
```
rdd = sc.parallelize(range(10))
```
<br>2. SparkContext textFile() method on a file:
```
rdd2 = sc.textFile('text.txt')
```





**Loading data in PySpark shell**

In PySpark, we express our computation through operations on distributed collections that are automatically parallelized across the cluster. In the previous exercise, you have seen an example of loading a list as parallelized collections and in this exercise, you'll load the data from a local file in PySpark shell.

In [0]:
# Load a local file into PySpark shell
lines = sc.textFile('movie_quotes.txt')

**Use of lambda() with map()**

The map() function in Python returns a list of the results after applying the given function to each item of a given iterable (list, tuple etc.). The general syntax of map() function is map(fun, iter). We can also use lambda functions with map(). The general syntax of map() function with lambda() is map(lambda <agument>:<expression>, iter). Refer to slide 5 of video 1.7 for general help of map() function with lambda().

In [46]:
my_list = list(range(1,11))

print(my_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [47]:
# Square each item in my_list using map() and lambda().
squrd_list = list(map(lambda x: x**2, my_list))

print(squrd_list)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


**Use of lambda() with filter():**

Another function that is used extensively in Python is the filter() function. The filter() function in Python takes in a function and a list as arguments. The general syntax of the filter() function is filter(function, list_of_input). Similar to the map(), filter() can be used with lambda() function. The general syntax of the filter() function with lambda() is filter(lambda <argument>:<expression>, list). Refer to slide 6 of video 1.7 for general help of the filter() function with lambda().

In [0]:
my_list2 = [10, 21, 31, 40, 51, 60, 72, 80, 93, 101]

In [49]:
# Filter the numbers divisible by 10 from my_list2 using filter() and lambda().

filtered_list2 = list(filter(lambda x: x % 10 == 0, my_list2))
print(filtered_list2)

[10, 40, 60, 80]


### Lesson 2: Programming in PySpark RDD’s

**RDDs** stand for _Resilient Distributed Dataset_, they are the first class citizen of Apache Spark.<br>It is simply a collection of data distributed across the cluster. RDD is the fundamental and backbone data type in Pyspark.

**Decomposing RDDs:**

Let's look at the different features of RDDs:
1. **_Resilient :_** Means the ability to withstand failures and recompute missing or damaged partitions.

2. **_Distributed :_** Means spanning the jobs across multiple nodes in the cluster for efficient computation.

3. **_Datasets :_** These are a collection of partitioned data e.g Arrays, Tuples, Tables, E.t.c.



**Creating RDDs:**

RDDs are created in 3 different ways:
1. Using the Spark Context .parallelize() method on an iterable like a list or range
```
rdd = sc.parallelize(range(100))
rdd2 = sc.parallelize('hello world')
```
2. creating RDDs from an external files like a text file or CSV file. This is by far the most common way to create RDDs in Pyspark. This method uses the Spark Context's textFile() method. Here we can read a README.md file stored locally on our computer.
```
rdd2 = sc.textFile('README.md')
```
3. Creating RDDs from other RDDs


**Understanding Partitioning in Pyspark**

Understanding how Spark deals with partitions allows one to control parallelism
. A partition in Spark is the division of the large dataset, with each part being stored in multiple locations across the cluster. By default Spark partitions the Data at the time of creating RDDs, based on several factors such as available resources, external datasets e.t.c

However this behavior can be controlled when we create an RDD via textFile() by passing a second argument called minPartitions, which specifies the minimum number of partitions to be created for an RDD.

In the parallelize() method, no need to use the keyword instead just pass the number of partitions you want as the second argument


In [50]:
rdd1 = sc.parallelize(range(100), 6)
# Let's see the type of rdd1
type(rdd1)

pyspark.rdd.PipelinedRDD

In [51]:
# let's see the number of partitions in rdd1
rdd1.getNumPartitions()

6

**RDDs from Parallelized collections**

Resilient Distributed Dataset (RDD) is the basic abstraction in Spark. It is an immutable distributed collection of objects. Since RDD is a fundamental and backbone data type in Spark, it is important that you understand how to create it. In this exercise, you'll create your first RDD in PySpark from a collection of words.

In [52]:
RDD = sc.parallelize(["Spark", "is", "a", "framework", "for", "Big Data processing"])
RDD.collect()

['Spark', 'is', 'a', 'framework', 'for', 'Big Data processing']

In [53]:
type(RDD)

pyspark.rdd.RDD

**RDDs from External Datasets**

PySpark can easily create RDDs from files that are stored in external storage devices such as HDFS (Hadoop Distributed File System), Amazon S3 buckets, etc. However, the most common method of creating RDD's is from files stored in your local file system. This method takes a file path and reads it as a collection of lines. In this exercise, you'll create an RDD from the file path

In [55]:
from google.colab import files
uploaded = files.upload()

Saving movie_quotes.txt to movie_quotes.txt


In [56]:
file_rdd = sc.textFile('movie_quotes.txt')
# Let's see the first 6 items
file_rdd.take(6)

["1.Frankly, my dear, I don't give a damn--Gone With the Wind, 1939",
 '',
 "2.I'm going to make him an offer he can't refuse.--The Godfather, 1972",
 '',
 "3.You don't understand! I coulda had class. I coulda been a contender. I could've been somebody, instead of a bum, which is what I am.--On the Waterfront, 1954",
 '']

In [57]:
type(file_rdd)

pyspark.rdd.RDD

In [58]:
file_rdd.getNumPartitions()

1

**Partitions in your data**

SparkContext's textFile() method takes an optional second argument called minPartitions for specifying the minimum number of partitions. In this exercise, you'll create an RDD named fileRDD_part with 5 partitions and then compare that with fileRDD that you created in the previous

In [0]:
fileRDD_part = sc.textFile('movie_quotes.txt', minPartitions=5)

In [60]:
type(fileRDD_part)

pyspark.rdd.RDD

In [61]:
fileRDD_part.getNumPartitions()

5

**Overview of Pyspark Operations**

RDDs in Pyspark support two different types of operations:
1. **Transformations :** Transformations are operations on an RDD that return a new RDD
2. **Actions :** Are operations that perform some computation on an RDD.

**RDD Transformations:**

Transformations follow lazy evaluation
<br>Lazy evaluations denote that Spark creates a graph from all the operations we perform on an RDD. And execution of the graph only starts when an action is performed on an RDD. This is called **_lazy evaluation_** in Spark.

**The RDD Transformations we will look at are :**
1. map()
2. filter()
3. flatmap()
4. union()

_map() transformation :_ Takes in a function and applies it to each element in an RDD, with the result of the func being the new value of each element in the resulting RDD.

_filter() transformation :_ Takes in a function and returns an RDD that only has elements that pass the filter condition.

_flatMap() transformation :_ This is like map() but with the exception that it returns multiple values for each element in the source RDD.

_union() transformation :_ returns the union of one RDD with another RDD

In [62]:
# Example of flatMap() transformation
rdd = sc.parallelize(['Hello world','How are you?'])
flat_map_rdd = rdd.flatMap(lambda x: x.split(' '))

flat_map_rdd.collect()

['Hello', 'world', 'How', 'are', 'you?']

In [0]:
# Example of union() transformation...

# Let's define an input RDD to filter other RDDs from
inputRDD = sc.textFile('movie_quotes.txt')

# Next let's filter out 2 other RDDs
fuckRDD = inputRDD.filter(lambda x: 'fuck' in x)
youRDD = inputRDD.filter(lambda x: 'you' in x)

In [64]:
fuckRDD.collect()

["13.I'm not gonna die. Because today, I'm going to kill the motherfucker!",
 "15.You stupid fucking cunt. You, Williamson, I'm talking to you, shithead. You just cost me $6,000. Six thousand dollars, and one Cadillac.",
 "19.I'm just a fucked-up girl who's lookin for my own peace of mind; don't assign me yours."]

In [65]:
youRDD.collect()

['7.May the Force be with you.--Star Wars, 1977',
 "8.Fasten your seatbelts. It's going to be a bumpy night.--All About Eve, 1950",
 "9.Love means never having to say you're sorry.--Love Story, 1970",
 '10.Get your stinking paws off me, you damned dirty ape.--Planet of the Apes, 1968',
 "15.You stupid fucking cunt. You, Williamson, I'm talking to you, shithead. You just cost me $6,000. Six thousand dollars, and one Cadillac.",
 "19.I'm just a fucked-up girl who's lookin for my own peace of mind; don't assign me yours.",
 '20.Sit your five-dollar ass down before I make change']

In [66]:
# next lets union or join the fuckRDD to the youRDD
combinedRDD = fuckRDD.union(youRDD)

# Let's see it
combinedRDD.collect()

["13.I'm not gonna die. Because today, I'm going to kill the motherfucker!",
 "15.You stupid fucking cunt. You, Williamson, I'm talking to you, shithead. You just cost me $6,000. Six thousand dollars, and one Cadillac.",
 "19.I'm just a fucked-up girl who's lookin for my own peace of mind; don't assign me yours.",
 '7.May the Force be with you.--Star Wars, 1977',
 "8.Fasten your seatbelts. It's going to be a bumpy night.--All About Eve, 1950",
 "9.Love means never having to say you're sorry.--Love Story, 1970",
 '10.Get your stinking paws off me, you damned dirty ape.--Planet of the Apes, 1968',
 "15.You stupid fucking cunt. You, Williamson, I'm talking to you, shithead. You just cost me $6,000. Six thousand dollars, and one Cadillac.",
 "19.I'm just a fucked-up girl who's lookin for my own peace of mind; don't assign me yours.",
 '20.Sit your five-dollar ass down before I make change']

**RDD Actions:**

After transformations, we usually need to do some action in the RDD. Remember RDDs are inherently lazy and must be actioned to derive any outputs. 
<br>The Actions we would use here are:
1. **collect() :_** returns all elemnets in the RDD
2. **take(N) :_** returns the N number of elements from the RDD
3. **first() :_** returns the first element and is similar to take(1)
4. **count() :_** returns the total number of elements in the RDD


**Map and Collect :**

The main method by which you can manipulate data in PySpark is using map(). The map() transformation takes in a function and applies it to each element in the RDD. It can be used to do any number of things, from fetching the website associated with each URL in our collection to just squaring the numbers. In this simple exercise, you'll use map() transformation to cube each element in an RDD

In [67]:
rdd = sc.parallelize(range(0,101,10))

# use map to cube each element in rdd
cubed = rdd.map(lambda x: x**3)
cubed.collect()

[0, 1000, 8000, 27000, 64000, 125000, 216000, 343000, 512000, 729000, 1000000]

**Filter and Count:**

The RDD transformation filter() returns a new RDD containing only the elements that satisfy a particular function. It is useful for filtering large datasets based on a keyword. For this exercise, you'll filter out lines containing keyword Spark from fileRDD RDD which consists of lines of text from the README.md file. Next, you'll count the total number of lines containing the keyword Spark and finally print the first 4 lines of the filtered RDD.

In [68]:
# Let's grab the spark README.md file using wget
!rm README.md* -f
!wget https://raw.githubusercontent.com/carloapp2/SparkPOT/master/README.md

--2020-03-20 00:28:48--  https://raw.githubusercontent.com/carloapp2/SparkPOT/master/README.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3624 (3.5K) [text/plain]
Saving to: ‘README.md’

README.md           100%[===================>]   3.54K  --.-KB/s    in 0s      

2020-03-20 00:28:48 (104 MB/s) - ‘README.md’ saved [3624/3624]



In [69]:
file_rdd = sc.textFile('README.md')
file_rdd.take(5)

['# Apache Spark',
 '',
 'Spark is a fast and general cluster computing system for Big Data. It provides',
 'high-level APIs in Scala, Java, and Python, and an optimized engine that',
 'supports general computation graphs for data analysis. It also supports a']

In [70]:
# Create filter() transformation to select only the lines containing the keyword Spark.
fileRDD_filter = file_rdd.filter(lambda x: 'Spark' in x)
fileRDD_filter.collect()

['# Apache Spark',
 'Spark is a fast and general cluster computing system for Big Data. It provides',
 'rich set of higher-level tools including Spark SQL for SQL and structured',
 'and Spark Streaming for stream processing.',
 'You can find the latest Spark documentation, including a programming',
 '## Building Spark',
 'Spark is built using [Apache Maven](http://maven.apache.org/).',
 'To build Spark and its example programs, run:',
 '["Building Spark"](http://spark.apache.org/docs/latest/building-spark.html).',
 'The easiest way to start using Spark is through the Scala shell:',
 'Spark also comes with several sample programs in the `examples` directory.',
 '    ./bin/run-example SparkPi',
 '    MASTER=spark://host:7077 ./bin/run-example SparkPi',
 'Testing first requires [building Spark](#building-spark). Once Spark is built, tests',
 '[run all automated tests](https://cwiki.apache.org/confluence/display/SPARK/Contributing+to+Spark#ContributingtoSpark-AutomatedTesting).',
 'Spark u

In [71]:
print("The total number of lines with the keyword Spark is", fileRDD_filter.count())

The total number of lines with the keyword Spark is 19


**<h3>Pair-RDDs in PySpark:</h3>**

Working with RDDs of key/Value pairs, which are a common data type for many operations in Spark.
<br>Real life datasets are usually Key/Value pairs. Each row is a key that maps to one or more values.
<br>In order to deal with this kind of data set, Pyspark provides a special kind of data structure called _Pair-RDDs._ In Pair-RDDs, the key refers to the identifier, while the value refers to the data.


**Creating Pair-RDDs:-**

Two common ways to create Pair-RDDs
1. From a list of Key/Value tuples
2. From a regular RDD

Irrespective of the method, the first step to create Pair-RDD is getting the data into Key/Value form.

Example 1: Creating a Pair-RDD from a list of tuples

In [0]:
# A list of tuples
my_tuple = [('Sam',23),('Tim',46),('May',33),('John', 54)]

# Creating a Pair RDD
pairRDD_tuple = sc.parallelize(my_tuple)

In [73]:
# Let's see the first element
pairRDD_tuple.first()

('Sam', 23)

In [74]:
type(pairRDD_tuple)

pyspark.rdd.RDD

Example 2: Creating a Pair-RDD from a regular RDD

In [78]:
my_list = ['Sam 23','Tim 46','May 33','John 54']
regular_rdd = sc.parallelize(my_list)
regular_rdd.take(1)

['Sam 23']

In [80]:
# Now let's create a pair RDD drom the regualr_rdd
pairRDD_regular = regular_rdd.map(lambda x: (x.split(' ')[0], int(x.split(' ')[1])))
pairRDD_regular.collect()

[('Sam', 23), ('Tim', 46), ('May', 33), ('John', 54)]

**Transformations on Pair-RDDs:**

Pair-RDDs are RDDS, so all transformations applicable to regular RDDs are applicable to them.
<Br>Since Pair-RDDs contain tuples, we need to pass functions that operate on Key/Value pairs.
<br>A few special operations are available for this kind, such as:
1. reduceByKey(func): Combines values with the same key.
2. groupByKey(): Group values with the same key.
3. sortByKey(): Return an RDD sorted by the key.
4. join(): Join two Pair-RDDs based on their key.

**reduceByKey() Transformation:**

This is the most popular Pair-RDD transformation, which combines values of the same key using a function.
<br>reduceByKey() runs several parallel operations, one for each key in the dataset. reduceByKey() returns a new RDD consisting of each key and the reduced value for that key.

For example let's combine or add the goals scored for each player in the list of tuples below

In [81]:
tup_list = [('Messi',23), ('Ronaldo',34), ('Neymar',22), ('Messi',24)]

# Next we create a regular RDD and chain it to the reduceByKey() transformation
# In one line of code, creating an RDD with total goals scored per player.
tup_list_reduceByKey = sc.parallelize(tup_list).reduceByKey(lambda x,y: x + y)

# Finally display all key-value pairs of the new RDD
tup_list_reduceByKey.collect()

[('Messi', 47), ('Ronaldo', 34), ('Neymar', 22)]

**sortByKey() Transformation:**

Sorting of data is important for many applications. We can sort Pair-RDDs as long as there's an order defined in the keys.
<br>This transformation returns an RDD, sorted by keys in ascending or descending order.

Using the previous example, let's sort the last RDD by descending number of goals scored per player.

In [82]:
# First let's reverse the RDD making goals scored the key, player name the value
tup_list_reduceByKey_reversed = tup_list_reduceByKey.map(lambda x: (x[1], x[0]))

# Now let's sort it by descending order
tup_list_reduceByKey_reversed.sortByKey(ascending=False).collect()

[(47, 'Messi'), (34, 'Ronaldo'), (22, 'Neymar')]

**groupByKey() Transformation:**

This allows us to group the data by key. It groups all the values with the same key in the Pair-RDD. For example grouping all of the airports for a particular country together.

In [87]:
# Example of groupByKey() transformation

airports_tups = [('US','JFK'), ('UK','LHR'), ('FR','CDG'), ('US','SFO')]

airports_tups_groupd = sc.parallelize(airports_tups).groupByKey().collect()

for cont, airport in airports_tups_groupd:
    print(cont,list(airport))

US ['JFK', 'SFO']
UK ['LHR']
FR ['CDG']


**join() Transformation:**

This joins two Pair-RDDs based on their key

In [0]:
# First let's create 3 lists of players, goals and countries
players = ['Messi','Ronaldo','Neymar']
goals_scored = [34, 32, 24]
origin_country = ['Argentina','Portugal','Brazil']

# Next using zip to zip 2 lists per time from above together to form 2 Pair-RDDs
players_goal_rdd = sc.parallelize(list(zip(players,goals_scored)))
players_country_rdd = sc.parallelize(list(zip(players, origin_country)))

In [97]:
# Next let's join the 2 Pair-RDDs based on their key to get an RDD that shows,
# The name as key and goals scored and country as values for each player above.

players_goal_rdd.join(players_country_rdd).collect()

[('Ronaldo', (32, 'Portugal')),
 ('Messi', (34, 'Argentina')),
 ('Neymar', (24, 'Brazil'))]